In [46]:
parentfolder = 'D:\\Finn\\KPUM_NODDI'
inputdicombasefolder = 'DICOM_KPUM_NODDI\\sourcedata'
outputniftibasefolder = 'Data\\nifti'

# User input
subject = '0XX'
session = 'MR1'

# Define good DCM datafolder as input
# If the data is not good enough, leave blank
dki_dirAP = 's17_DKI_2.5mm_NODDI_dir-AP_79dir_0_1000_2000'
dki_dirPA = 's16_DKI_2.5mm_NODDI_dir-PA_2dir_0'
t2w_mcrib = 's07_T2W_MCRIB'
t1w_mprage = ''

In [47]:
# Convert DICOM to NIFTI using dcm2niix

import os

class niftioutput:
  def __init__(self, dicomfolder, niftifolder, filebase):
    self.dicomfolder = dicomfolder
    self.niftifolder = niftifolder
    self.filebase = filebase

def dicom2nifti(in_path, out_path, out_filebase):
    # Anonymises input DCM according to data_elements in list
    import os, subprocess
    if os.path.isdir(in_path): 
        if not os.path.exists(out_path): # then make this first directory
           os.makedirs(out_path)
        p=subprocess.Popen(f"dcm2niix -a y -d 1 -b y -w 1 -o {out_path} -f {out_filebase} {in_path}", stdout=subprocess.PIPE, shell=True)  
    else :
       print(f'Input folder {in_path} does not exist, no conversion')


# Define input and output parameters for dcm2niix
inputdicomdir = os.path.join(parentfolder, inputdicombasefolder, f'sub-{subject}', f'ses-{session}')
outputniftidir = os.path.join(parentfolder, outputniftibasefolder, f'sub-{subject}', f'ses-{session}')
if not os.path.exists(outputniftidir): # then make this directory
    os.makedirs(outputniftidir)

# Now convert for different data types

# DKI dir-AP data
dki_dirAP = niftioutput(os.path.join(inputdicomdir, dki_dirAP), os.path.join(outputniftidir, 'dwi'), f"sub-{subject}_ses-{session}_dir-AP_dwi")
dicom2nifti(dki_dirAP.dicomfolder, dki_dirAP.niftifolder, dki_dirAP.filebase)
# DKI dir-PA data
dki_dirPA = niftioutput(os.path.join(inputdicomdir, dki_dirPA), os.path.join(outputniftidir, 'fmap'), f"sub-{subject}_ses-{session}_dir-PA_epi")
dicom2nifti(dki_dirPA.dicomfolder, dki_dirPA.niftifolder, dki_dirPA.filebase)
# T2w MCRIB data
t2w_mcrib = niftioutput(os.path.join(inputdicomdir, t2w_mcrib), os.path.join(outputniftidir, 'anat'), f"sub-{subject}_ses-{session}_acq-mcrib_T2w")
dicom2nifti(t2w_mcrib.dicomfolder, t2w_mcrib.niftifolder, t2w_mcrib.filebase)
# T1w MPRAGE data
t1w_mprage = niftioutput(os.path.join(inputdicomdir, t1w_mprage), os.path.join(outputniftidir, 'anat'), f"sub-{subject}_ses-{session}_acq-mprage_T1w")
dicom2nifti(t1w_mprage.dicomfolder, t1w_mprage.niftifolder, t1w_mprage.filebase)

OSError: [WinError 123] Felaktig syntax för filnamn, katalognamn eller volymetikett: 'D:\\Finn\\KPUM_NODDI\\Data\nifti'